In [ ]:
import g4f
from aiogram import Bot, Dispatcher, types
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import Command
from aiogram.types import Message
import asyncio
import os 

TOKEN = os.getenv("BOT_TOKEN")


# История сообщений
TG_CHATS = {}

def add_new_user(user_id):
    if user_id not in TG_CHATS:
        TG_CHATS[user_id] = []

def get_messages_from_history(user_id):
    if user_id not in TG_CHATS:
        add_new_user(user_id)
    return TG_CHATS[user_id]

def clear_history(user_id):
    if user_id in TG_CHATS:
        TG_CHATS[user_id] = []

def get_answer_from_ai(role, history, prompt):
    messages = [{"role": "system", "content": role}]
    messages.extend(history)
    messages.append({"role": "user", "content": prompt})
    return g4f.ChatCompletion.create(
        model=g4f.models.gpt_4o_mini,
        messages=messages
    )

def get_text_prompt_image(prompt):
    response = g4f.ChatCompletion.create(
        model=g4f.models.gpt_4o,
        messages=[
            {"role": "system", "content": "Ты помощник в генерации изображений. Ты внимательно слушаешь и подмечаешь все мелкие детали, тем самым, помогаешь писать промпт так, чтобы изображение полностью описывало текст. Ответ должен быть кратким, но понятным и точным."},
            {"role": "user", "content": prompt}
        ]
    )
    return response


def generate_image_from_text(prompt):
    try:
        # Генерирация промпта
        text_prompt = get_text_prompt_image(prompt)

        if isinstance(text_prompt, dict) and "choices" in text_prompt:
            final_prompt = text_prompt["choices"][0]["message"]["content"]
        else:
            final_prompt = str(text_prompt)

        print("Сгенерированный промпт для изображения:", final_prompt)

        client = g4f.client.Client()
        response = client.images.generate(
            model="dall-e-3",
            prompt=final_prompt,
            response_format="url"
        )

        print("Ответ от генерации изображения:", response)

        return response.data[0].url

    except Exception as e:
        print("Ошибка при генерации изображения:", e)
        return None

tg_bot = Bot(token=TOKEN, default=DefaultBotProperties(
    parse_mode=ParseMode.MARKDOWN
))
dp_tg = Dispatcher()

tg_role = (
    "Ты — весёлый и слегка чудаковатый сказочник по имени Бормотун. "
    "Твои сказки — это смесь доброй несуразицы, тонкого юмора, странных ситуаций и неожиданных поворотов. "
    "Ты любишь превращать обычные предметы и животных в героев: у тебя чайники разговаривают, козы открывают бизнес, а пингвины учатся танцевать балет. "
    "Несмотря на шутки и нелепости, каждая сказка несёт светлый смысл и добрую мораль, но подаётся она легко, весело и с улыбкой. "
    "Пиши живым, разговорным языком, можно с шутками и игрой слов. "
    "Если пользователь просит сказку — сразу выдумывай сюжет, не задавай встречных вопросов. "
    "Не пугай, не грусти — твоя задача: развеселить, удивить и подарить хорошее настроение."
)

@dp_tg.message(Command("start"))
async def send_hello_message_tg(message: Message):
    await message.reply("Я тут для того, чтобы рассказывать тебе сказки на ночь, под которые ты будешь засыпать сладко-сладко.")

@dp_tg.message()
async def just_send_answer_tg(message: Message):
    user_id = message.from_user.id
    message_text = message.text.strip()

    if message_text == "Расскажи сказку":
        message_text = "Придумай сказку на ночь"

    if message_text == "Сгенерируй картинку":
        message_text = "Придумай красивую сказку и опиши её так, чтобы по ней можно было нарисовать картину"

    history = get_messages_from_history(user_id)
    if len(history) > 50:
        clear_history(user_id)
        history = []

    # ответ от gpt
    answer_ai = get_answer_from_ai(tg_role, history, message_text)

    # Добавение в историю ответа и запроса
    history.append({"role": "user", "content": message_text})
    history.append({"role": "assistant", "content": answer_ai})

    # Генерация изобржения
    try:
        image_from_ai = generate_image_from_text(answer_ai)
        await message.answer_photo(image_from_ai, caption=answer_ai)
    except Exception as e:
        await message.reply(f"{answer_ai}\n\n Не удалось отправить изображение: {e}")
async def main():
    await dp_tg.start_polling(tg_bot)


task = main()
await task



